## **7. 변형 회귀 (Variant Regression)**
: 선형 회귀가 갖는 다양한 한계를 보완하기 위해 확장된 회귀 모델들

선형 회귀(OLS) 가정
- 선형성(linearity)
- 정규성(normality)
- 등분산성(homoscedasticity)
- 독립성(independence)

하지만 실제 데이터는 이러한 가정을 자주 위반 \
변수 개수가 많거나 비선형 패턴 존재  
이를 해결하기 위해 여러 변형 회귀 기법(Variant Regression)을 사용

## **필요한 라이브러리 가져오기**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, QuantileRegressor, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrix

import warnings
warnings.filterwarnings("ignore")

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["font.size"] = 11

### **데이터 불러오기**

In [ ]:
df = pd.read_csv('../data/california_housing.csv')

print("\n데이터 셋의 일부를 확인해보면 아래와 같습니다.")
df.head()

<html>
<b><font size=4>* 변수에 대한 설명</font></b><br><br>

- MedInc : 지역의 중위소득 (Median Income)<br>
- HouseAge : 주택의 평균 연식 (Average House Age)<br>
- AveRooms : 가구당 평균 방(room) 수<br>
- AveBedrms : 가구당 평균 침실 수<br>
- Population : 지역 인구수<br>
- AveOccup : 가구당 평균 거주자 수<br>
- Latitude : 위도<br>
- Longitude : 경도<br>
- MedHouseVal : 지역의 중위 주택가격 (Target, 단위: 10만 달러)<br>
</html>

### **데이터 전처리**
- 데이터 결측치 확인

In [ ]:
df.isnull().sum()

- 데이터 중복 확인

In [ ]:
df.duplicated().sum()

- describe() 함수를 사용해서 수치형 변수들만을 기준으로 카운트, 평균, 표준편차, 최소/최댓값, 4분위 수를 확인

In [ ]:
df.describe()

### **데이터 시각화**
- 변수 간 상관 관계 파악

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Feature Correlation Matrix')
plt.show()

- 각 특성의 분포에 대한 히스토그램

In [ ]:
df.hist(bins=15, figsize=(15, 10), layout=(4, 3))
plt.suptitle('Feature Distribution')
plt.show()

### **데이터 분리**
- 데이터를 학습 데이터 세트와 테스트 데이터 세트로 분리

In [ ]:
# 독립 변수(X)와 종속 변수(y)를 정의
X = df.drop("MedHouseVal", axis=1)
y = df["MedHouseVal"]

- 데이터의 80%는 학습에 사용되고 20%는 테스트에 사용

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("\n학습 세트 크기:", X_train.shape[0])
print("테스트 세트 크기:", X_test.shape[0])

- Feature Scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 스케일링 후 DataFrame으로 복원
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled  = pd.DataFrame(X_test_scaled,  columns=X_test.columns)

### **Linear Regression**
- LinearRegression 모델 생성 및 학습

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

- 테스트 데이터에 대한 가격 예측

In [ ]:
y_pred = lr_model.predict(X_test_scaled)

print("\n모델 계수(기울기):", lr_model.coef_[0])
print("모델 절편(y절편):", lr_model.intercept_)

- MAE, MSE, R2 값을 비교하여 모델을 평가

In [ ]:
mae_housing = mean_absolute_error(y_test,y_pred)
mse_housing = mean_squared_error(y_test,y_pred)
r2_housing = r2_score(y_test,y_pred)

print("\n[모델 평가 지표]")
print("\n- 평균 절대 오차 (MAE):", mae_housing)
print("- 평균 제곱 오차 (MSE):", mse_housing)
print("- R-제곱 (R2) 값:", r2_housing)

### **7-1 Stepwise Regression**


- 불필요한 변수를 제거하고 중요한 변수만 선택하는 회귀 방식
- **Forward Selection**: 유의한 변수를 하나씩 추가  
- **Backward Elimination**: 덜 유의한 변수를 하나씩 제거  
- **Bidirectional**: 두 방법을 동시에 적용하여 균형 잡힌 선택 수행  
- 기준: **교차검증(CV) RMSE** 
- 목적: **불필요한 변수를 줄이고, 해석 가능한 간결한 모델** 확보  
- 장점:  
  - 많은 후보 변수 중 핵심 변수 자동 선택  
  - 모델 복잡도 감소  
- 단점:  
  - 탐욕적(greedy) → 전역 최적 보장 안 됨  
  - 다중공선성(multicollinearity)에 취약  
  - 변수 선택 과정에서 과적합 위험 존재

## **7-1-1. 전진선택법 (Forward Selection)**

- **Forward Selection**: 유의한 변수를 하나씩 추가  

In [ ]:
import matplotlib.pyplot as plt

# 가정: forward_selection 함수에서 성능(metric)과 선택된 변수를 반환하도록 수정
def forward_selection_with_visualization(X, y, significance_level=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    selected_features = []
    model_performance = []  # 모델 성능 추적 (예: R^2 또는 AIC)

    while remaining_features:
        best_pval = float("inf")
        best_feature = None
        for feature in remaining_features:
            X_subset = sm.add_constant(X[initial_features + [feature]])
            model = sm.OLS(y, X_subset).fit()
            pval = model.pvalues[feature]
            if pval < best_pval and pval < significance_level:
                best_pval = pval
                best_feature = feature
        if best_feature:
            initial_features.append(best_feature)
            remaining_features.remove(best_feature)
            selected_features.append(best_feature)
            # 모델 성능 저장
            X_subset = sm.add_constant(X[initial_features])
            model = sm.OLS(y, X_subset).fit()
            model_performance.append(model.rsquared)  # R^2 값 저장
        else:
            break
    return selected_features, model_performance

selected_features, model_performance = forward_selection_with_visualization(X, y)

print("전진 선택법 이후 선택된 변수들 :", selected_features)
print("전진 선택에 따른 모델 성능:", model_performance)


In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(model_performance) + 1), model_performance, marker='o', linestyle='-')
plt.xticks(range(1, len(model_performance) + 1), selected_features, rotation=45)
plt.xlabel('Selected Features (in order)')
plt.ylabel('Model Performance (R^2)')
plt.title('Forward Selection: Feature Addition and Model Performance')
plt.grid()
plt.show()

## **7-1-2. 후진제거법 (Backward Elimination)**

- **Backward Elimination**: 덜 유의한 변수를 하나씩 제거  

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

# 후진 소거법 구현
def backward_elimination_with_visualization(X, y, significance_level=0.05):
    features = list(X.columns)  # 모든 독립 변수 포함
    model_performance = []  # 모델 성능 기록 (R^2)

    while len(features) > 0:
        # 현재 변수로 회귀 모델 생성
        X_subset = sm.add_constant(X[features])  # 상수항 추가
        model = sm.OLS(y, X_subset).fit()

        # R^2 기록
        model_performance.append(model.rsquared)

        # p-value가 가장 높은 변수 찾기
        pvalues = model.pvalues[1:]  # 상수(constant)는 제외
        max_pval = pvalues.max()

        if max_pval > significance_level:  # 유의수준 초과 시 변수 제거
            excluded_feature = pvalues.idxmax()
            features.remove(excluded_feature)
        else:
            break  # 모든 변수의 p-value가 유의수준 이하일 경우 종료

    return features, model_performance

selected_features_backward, model_performance_backward = backward_elimination_with_visualization(X, y)

print("후진 제거법 이후 선택된 변수들 :", selected_features_backward)
print("후진 제거에 따른 모델 성능:", model_performance_backward)


In [ ]:
plt.figure(figsize=(8, 6))

x_ticks = range(len(model_performance_backward))

# 모델 성능 그래프
plt.plot(x_ticks, model_performance_backward, marker='o', linestyle='-', color='b', label='R^2')

# x축 레이블 (남아있는 변수 이름) 길이 조정
adjusted_features = selected_features_backward[::-1][:len(model_performance_backward)]
plt.xticks(x_ticks, adjusted_features, rotation=45)

plt.xlabel('Remaining Features (in order of removal)')
plt.ylabel('Model Performance (R^2)')
plt.title('Backward Elimination: Feature Removal and Model Performance')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


## **7-1-3. Bidirectional**

- **Bidirectional**: 두 방법을 동시에 적용하여 균형 잡힌 선택 수행  

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

def bidirectional_stepwise_with_visualization(X, y, significance_level_in=0.05, significance_level_out=0.05):
    selected_features = []              # 현재 선택된 변수 집합
    remaining_features = list(X.columns)  # 선택되지 않은 변수들
    model_performance = []              # 각 단계에서의 모델 성능 기록 (R²)

    while True:
        changed = False

        # -----------------------------
        # 1) Forward Selection Step
        # -----------------------------
        best_pval = float("inf")
        best_feature = None

        for feature in remaining_features:
            X_subset = sm.add_constant(X[selected_features + [feature]])
            model = sm.OLS(y, X_subset).fit()

            pval = model.pvalues[feature]

            if pval < best_pval and pval < significance_level_in:
                best_pval = pval
                best_feature = feature

        if best_feature is not None:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            changed = True

            # 모델 성능 저장
            X_subset = sm.add_constant(X[selected_features])
            model = sm.OLS(y, X_subset).fit()
            model_performance.append(model.rsquared)

        # -----------------------------
        # 2) Backward Elimination Step
        # -----------------------------
        if len(selected_features) > 0:
            X_subset = sm.add_constant(X[selected_features])
            model = sm.OLS(y, X_subset).fit()

            # 상수항 제외
            pvalues = model.pvalues.iloc[1:]

            worst_pval = pvalues.max()
            worst_feature = pvalues.idxmax()

            if worst_pval > significance_level_out:
                selected_features.remove(worst_feature)
                remaining_features.append(worst_feature)
                changed = True

                # 모델 성능 저장
                X_subset = sm.add_constant(X[selected_features]) if selected_features else None
                if selected_features:
                    model = sm.OLS(y, X_subset).fit()
                    model_performance.append(model.rsquared)
                else:
                    model_performance.append(None)

        # -----------------------------
        # 종료 조건
        # -----------------------------
        if not changed:
            break

    return selected_features, model_performance

# 함수 실행
selected_features_bidir, performance_bidir = bidirectional_stepwise_with_visualization(X, y)

print("Bidirectional Stepwise 이후 선택된 변수들 :", selected_features_bidir)
print("Bidirectional Stepwise 성능 변화:", performance_bidir)

In [ ]:

plt.figure(figsize=(8, 6))
plt.plot(range(1, len(performance_bidir) + 1), performance_bidir,
         marker='o', linestyle='-', color='green')
plt.xticks(range(1, len(performance_bidir) + 1), selected_features_bidir + ([""] * (len(performance_bidir) - len(selected_features_bidir))), rotation=45)

plt.xlabel("Feature Added/Removed (Step Order)")
plt.ylabel("Model Performance (R²)")
plt.title("Bidirectional Stepwise: Feature Selection and Model Performance")
plt.grid()
plt.tight_layout()
plt.show()

### **7-2. Quantile Regression**

\begin{equation}
Q_\tau(Y|X) = \beta_{\tau,0} + \beta_{\tau,1} x_1 + \cdots + \beta_{\tau,p} x_p
\end{equation}

- 평균이 아니라 **조건부 분위수(quantile)**를 예측하는 회귀
- 평균(OLS)만 설명하는 대신, **분포의 여러 지점(예: 0.1, 0.5, 0.9 분위)** 을 직접 추정  
- **Pinball(Quantile) Loss** 최소화를 통해 학습  
- 장점:  
  - **비대칭 분포**나 **꼬리(heavy tail)** 특성을 반영 가능  
  - **이상치(outlier)에 강건** (특히 median regression, q=0.5)  
  - 중앙값, 하위/상위 분위 등 **다양한 조건부 경향** 분석 가능  
- 단점:  
  - 평균 기준 지표(MSE, R²)에서는 OLS보다 불리하게 보일 수 있음  
  - 분위 간 **교차(crossing)** 문제 발생 가능  

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import QuantileRegressor


# --------------------
# 3) Quantile Regression (MedInc만 사용)
# --------------------
feature = "MedInc"

X_train_medinc = X_train_scaled[[feature]]
X_test_medinc  = X_test_scaled[[feature]]

# x축 범위 (원본 값)
x_min = X[feature].min()
x_max = X[feature].max()
x_range_original = np.linspace(x_min, x_max, 200).reshape(-1, 1)

# x_range_original -> scaled 변환
x_range_scaled = scaler.transform(
    pd.DataFrame(np.hstack([x_range_original,
                            np.zeros((200, X_train.shape[1]-1))]),
                 columns=X.columns)
)
x_range_scaled_medinc = x_range_scaled[:, X.columns.get_loc(feature)].reshape(-1, 1)


- 시각화

In [ ]:
# plot
plt.figure(figsize=(8, 5))
plt.scatter(X[feature], y, alpha=0.2)

quantiles = [0.25, 0.5, 0.75]

for q in quantiles:
    model_q = QuantileRegressor(quantile=q, alpha=0.001)
    model_q.fit(X_train_medinc, y_train)

    y_q_pred = model_q.predict(x_range_scaled_medinc)
    plt.plot(x_range_original, y_q_pred, label=f"tau={q}")

plt.xlabel("MedInc")
plt.ylabel("MedHouseVal")
plt.title("Quantile Regression (MedInc → MedHouseVal)")
plt.legend()
plt.grid()
plt.show()

### **7-3. Bayesian Regression**

- 회귀 계수를 **점추정**이 아닌 **확률분포**로 추정하는 모델
- **불확실성까지 예측** : 예측값뿐 아니라 표준편차/예측구간(PI) 제공
- **사전(prior)으로 수축** : 작은 표본·다중공선성에서 과적합 완화
- **ARD (Automatic Relevance Determination)** : 불필요한 변수의 계수를 자동으로 0 근처로 수축
- 장점:
    - 작은 n / 큰 p, 강한 다중공선성에서도 안정적인 추정 가능
    - 계수·예측의 불확실성을 함께 제공
- 단점:
    - 가우시안/등분산 가정에 기반하여 이상치·비대칭 분포·이분산에 취약
    - 사전(prior) 의존성으로 인한 편향 가능
    - 하이퍼파라미터·evidence 해석이 직관적이지 않음

In [ ]:
from sklearn.linear_model import BayesianRidge

model_bayes = BayesianRidge()
model_bayes.fit(X_train_scaled, y_train)

# 예측 + 불확실성
y_mean, y_std = model_bayes.predict(X_test_scaled, return_std=True)

print("Bayesian Regression")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_mean)))
print("R^2 :", r2_score(y_test, y_mean))


- 시각화

In [ ]:
plt.figure(figsize=(10,4))
order = np.argsort(y_mean)

plt.errorbar(
    np.arange(len(y_mean))[order],
    y_mean[order],
    yerr=y_std[order],
    fmt="o",
    alpha=0.5
)
plt.title("Bayesian Regression Prediction Uncertainty")
plt.xlabel("Sorted Test Samples")
plt.ylabel("Predicted MedHouseVal")
plt.grid()
plt.show()

- 시각화

### **7-4. Generalized Linear Model (GLM)**

\begin{equation}
g(\mu) = X\beta
\end{equation}

- 종속변수가 정규분포가 아닐 때 사용하는 일반화된 회귀 프레임워크
- 다양한 분포(이항, 포아송 등) 지원  
- 링크 함수(link function)를 통해 분포와 선형식 연결
- **장점**
    - 카운트/율 데이터에 자연스러운 손실(편차)와 해석(계수 -> 비율 변화) 제공
    - 오프셋(log 노출 시간) 등 실무 확장 용이

- **단점**
    - 과산포(Var > μ), 제로-인플레이션이 크면 부적합 -> NB/Quasi/ZIP 고려
    - 링크 공간 비선형·상호작용 누락 시 편향, 반복측정 상관 시 GLMM/GEE 필요


In [ ]:
import statsmodels.formula.api as smf

formula_glm = "MedHouseVal ~ MedInc + AveRooms + AveOccup"

model_glm = smf.glm(
    formula=formula_glm,
    data=df,
    family=sm.families.Poisson()
).fit()

print(model_glm.summary())


### **7-5. Spline Regression**

- 하나의 직선이 아닌, **구간별(piecewise)** 다항함수로 비선형 관계 모델링
- 전체 구간에서 하나의 다항식(예: 다항 회귀)을 쓰는 대신, 특정 지점(**knots**)을 기준으로 구간을 나누어 각 구간마다 다항식을 적합
- 구간이 바뀌더라도 함수값과 기울기(미분값)가 매끄럽게 이어지도록 **연속성(continuity)** 조건을 부여
- Cubic Spline(3차 스플라인)이 가장 일반적으로 사용
- 장점:  
  - 비선형 패턴을 효과적으로 설명할 수 있음
  - 고차 다항식 회귀보다 안정적이고 과적합 위험이 적음
  - 국소적으로 유연하여 다양한 데이터 형태를 잘 포착


- 단점:
  - Knot의 위치와 개수를 임의로 정해야 하는 경우 다수 → 모델 선택 어려움
  - 구간이 많아질수록 모델이 복잡해지고 해석력 저해
  - Knot 위치를 잘못 선택하면 성능이 크게 저하

In [ ]:
from patsy import dmatrix
import statsmodels.api as sm

x_s = df["MedInc"]
y_s = df["MedHouseVal"]

# Spline basis 생성
transformed_x = dmatrix(
    "bs(x_s, df=6, degree=3)",
    {"x_s": x_s},
    return_type="dataframe"
)

# OLS 적합
model_spline = sm.OLS(y_s, transformed_x).fit()

# 예측 범위
x_range = np.linspace(x_s.min(), x_s.max(), 300)
x_range_bs = dmatrix(
    "bs(x_range, df=6, degree=3)",
    {"x_range": x_range},
    return_type="dataframe"
)
y_pred = model_spline.predict(x_range_bs)

plt.figure(figsize=(8, 5))
plt.scatter(x_s, y_s, alpha=0.3)
plt.plot(x_range, y_pred, color="red")
plt.title("Spline Regression (MedInc → MedHouseVal)")
plt.xlabel("MedInc")
plt.ylabel("MedHouseVal")
plt.grid()
plt.show()


### **7-6. Locally Weighted Regression (LWR / LOESS)**

- Weighted Regression(가중 회귀)은 각 데이터 포인트에 가중치(weight)를 부여하여 모델을 학습하는 방법  
- 모든 데이터가 동일한 중요도를 가진다고 가정하는 일반 회귀와 달리, 특정 구간이나 특정 데이터의 영향력을 줄이거나 높일 수 있음
- 데이터가 이질적이거나, 관측값의 신뢰도가 다를 때 효과적

- 장점
  - **이상치(outlier) 완화**: 이상치나 신뢰도 낮은 데이터의 가중치를 줄여 모델에 미치는 영향을 최소화할 수 있음  
  - **데이터 품질 반영**: 관측값의 정확도, 표본 추출 과정의 신뢰도 등에 따라 데이터를 차등 반영 가능  
  - **유연한 모델링**: 특정 구간의 패턴을 더 강조하거나, 특정 집단의 데이터에 집중할 수 있음  
  - **현실적 적용**: 실험 데이터, 설문 데이터처럼 각 관측치의 신뢰도가 다를 때 매우 유용


- 단점
  - **가중치 설정의 어려움**: 가중치를 어떻게 줄지 명확한 기준이 없으면 오히려 왜곡된 결과를 만들 수 있음  
  - **해석 복잡성**: 단순 OLS보다 결과 해석이 어렵고, 잘못된 가중치 부여는 편향(bias)을 심화시킬 수 있음  
  - **데이터 의존성**: 특정 구간의 데이터가 너무 적거나 가중치가 지나치게 낮으면 해당 구간은 거의 무시됨  
  - **계산 복잡성 증가**: 대규모 데이터에서 WLS(Weighted Least Squares) 적용 시 계산량이 늘어날 수 있음

In [ ]:
X_lw = df[["MedInc"]]
y_lw = df["MedHouseVal"]

knn = KNeighborsRegressor(n_neighbors=20, weights="distance")
knn.fit(X_lw, y_lw)

x_grid = np.linspace(X_lw.min(), X_lw.max(), 200).reshape(-1, 1)
y_grid = knn.predict(x_grid)

- 시각화

In [ ]:
plt.scatter(X_lw, y_lw, alpha=0.2)
plt.plot(x_grid, y_grid, color="red")
plt.title("Locally Weighted Regression (MedInc → MedHouseVal)")
plt.show()